In [16]:
#Importing relevant modules
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
%matplotlib inline
import scipy.optimize as sco
import requests
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import requests
from scipy.stats import norm
import math 

# General Functions

In [17]:
# Function to retrieve the current federal funds rate 
def get_current_interest_rate():
    api_url = 'https://api.stlouisfed.org/fred/series/observations'
    series_id = 'DFF'  # Series ID for the Effective Federal Funds Rate
    api_key = 'REPLACE'  # To be replaced (API keys to be found in google docs)

    # Setting the parameters for the API request
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
        'sort_order': 'desc',
        'limit': 1
    }
    # In case the rate is not retrievable we need an error message
    try:
        response = requests.get(api_url, params=params)
        data = response.json()
        interest_rate = data['observations'][0]['value']
        return float(interest_rate)
    except (requests.RequestException, KeyError, IndexError):
        return None

In [18]:
# The below functions are inspired by:
    # https://medium.com/analytics-vidhya/modern-portfolio-theory-model-implementation-in-python-e416facabf46
    # https://amangupta16.medium.com/portfolio-optimization-using-python-part-1-2-9fd80097a606
    # https://www.analyticsvidhya.com/blog/2021/04/portfolio-optimization-using-mpt-in-python/
    
# Function to calculate the annualized performance of a portfolio
def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    n = len(mean_returns)
    returns = np.sum(np.multiply(mean_returns, weights)) * 252  # Calculate the portfolio returns (Assumed 252 trading days)
    cov_weights = np.dot(cov_matrix, weights) # calculate weighted covariance of the portfolio
    std = np.sqrt(np.dot(weights, cov_weights)) * np.sqrt(252)  # Calculate the portfolio standard deviation
    return std, returns

# Function to generate random portfolios and calculate their performance
def random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    num_stocks = len(mean_returns)
    results = np.zeros((3, num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(num_stocks)  # Generate random weights for the portfolio
        weights /= np.sum(weights)  # Normalize the weights to sum up to 1
        weights_record.append(weights)  # Record the weights for each portfolio
        portfolio_std_dev, portfolio_return = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
        results[0, i] = portfolio_std_dev  # Store the portfolio standard deviation in the results
        results[1, i] = portfolio_return  # Store the portfolio returns in the results
        results[2, i] = (portfolio_return - risk_free_rate) / portfolio_std_dev  # Calculate the Sharpe Ratio and store in reults
    return results, weights_record

# Function to calculate the negative Sharpe ratio (to be minimized)
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_var, p_returns = portfolio_annualised_performance(weights, mean_returns, cov_matrix)  # Calculate the portfolio's annualized return and variance
    neg_sharpe = -(p_returns - risk_free_rate) / p_var  # Calculate the negative Sharpe Ratio
    return neg_sharpe

# Function to find the portfolio with the maximum Sharpe Ratio
def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)  # Set up the arguments for the neg_sharpe_ratio function
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]  # Set up the equality constraint with weights summing up to 1
    bounds = [(0.0, 1.0) for _ in range(num_assets)]  # Set up the bounds for each weight (between 1 and 0)
    result = sco.minimize(neg_sharpe_ratio, num_assets * [1.0 / num_assets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)  # Use the SLSQP optimization method to maximize the negative Sharpe ratio
    return result

# Function to calculate the portfolio annual volatility
def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[0]

# Function to find the portfolio with the minimum variance
def min_variance(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)  # Set up the arguments for the portfolio_volatility function
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]  # Set up the equality constraint with weights summing up to 1
    bounds = [(0.0, 1.0) for _ in range(num_assets)]  # Set up the bounds for each weight (between 1 and 0)
    result = sco.minimize(portfolio_volatility, num_assets * [1.0 / num_assets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)  # Use the SLSQP optimization method to minimize portfolio volatility
    return result

# Function to the efficient portfolio for a given target return.
def efficient_return(mean_returns, cov_matrix, target):
    num_assets = len(mean_returns) # Count the number of stocks
    args = (mean_returns, cov_matrix)
    
    # Function to calculate the annualized portfolio return.
    def portfolio_return(weights):
        return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[1]  # Calculate the portfolio's annualized performance and retrieve the returns
    
    constraints = [{'type': 'eq', 'fun': lambda x: portfolio_return(x) - target}, {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]  # Making sure portfolio return matches the target return and weights sum up to 1
    bounds = [(0, 1) for _ in range(num_assets)]  # Set up the bounds for each weight (between 1 and 0)
    result = sco.minimize(portfolio_volatility, num_assets * [1.0 / num_assets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)  # Use the SLSQP optimization method to find the efficient portfolio
    return result

# Function to generate the efficient frontier by calculating efficient portfolios for a range of target returns.
def efficient_frontier(mean_returns, cov_matrix, returns_range):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_returns, cov_matrix, ret))  # Calculate the efficient portfolio for each target return
    return efficients
 
# Function to display the calculated efficient frontier with random portfolios and specific scenarios
def display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate, scenario, money_to_invest):
    results, _ = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)
    
    # Calculate the portfolio with the maximum Sharpe ratio
    global sdp, rp
    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp, rp = portfolio_annualised_performance(max_sharpe['x'], mean_returns, cov_matrix)
    
    # Calculate the allocation for the maximum Sharpe ratio portfolio
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=pf_data.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation['allocation'] = (max_sharpe_allocation['allocation'] / 100)* money_to_invest
    max_sharpe_allocation = max_sharpe_allocation.T
    
    # Calculate the portfolio with the minimum volatility
    global sdp_min, rp_min
    min_vol = min_variance(mean_returns, cov_matrix)
    sdp_min, rp_min = portfolio_annualised_performance(min_vol['x'], mean_returns, cov_matrix)
   
    # Calculate the allocation for the minimum volatility portfolio
    min_vol_allocation = pd.DataFrame(min_vol.x,index=pf_data.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation['allocation'] = (min_vol_allocation['allocation'] / 100)* money_to_invest
    min_vol_allocation = min_vol_allocation.T
    
    # Determine the result based on the scenario
    if scenario == 1:
        return_variable = "-"*80 + "\n" + "Maximum Sharpe Ratio Portfolio Allocation" + "\n" + "Annualised Return:" + str(round(rp,2)) + "\n" + "Annualised Volatility:" + str(round(sdp,2)) + "\n" + str(max_sharpe_allocation)
    elif scenario == 2:
        return_variable = "-"*80 + "\n" + "Minimum Volatility Portfolio Allocation" + "\n" + "Annualised Return:" + str(round(rp_min,2)) + "\n" + "Annualised Volatility:" + str(round(sdp_min,2)) + "\n" + str(min_vol_allocation)
    
    # Create a scatter plot of the random portfolios and the optimized portfolios
    fig = plt.figure(figsize=(10, 12))
    norm = plt.Normalize(results[2, :].min(), results[2, :].max())
    plt.scatter(results[0, :], results[1, :], c=results[2, :], cmap=cm.Greys, norm=norm, marker='o', s=10, alpha=0.3)
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')
    
    # Generate the efficient frontier
    target = np.linspace(rp_min, 0.40, 50)
    efficient_portfolios = efficient_frontier(mean_returns, cov_matrix, target)
    
    # Set plot title, labels, and legend
    plt.title('Calculated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('Annualised Volatility')
    plt.ylabel('Annualised Returns')
    plt.legend(labelspacing = 0.8)
    return return_variable, fig

In [19]:
#Function that retreives the top 3 headlines for each stock in a list
def get_stock_headlines(stocks):
    base_url = "https://www.alphavantage.co/query" # Base URL for the API request
    list_of_headlines = "" # String to store the list of headlines
    # Iterate over each stock in the input list
    for stock in stocks: 
        # Set parameters for the API request
        params = {
            "function": "NEWS_SENTIMENT",
            "tickers": stock,
            "apikey": "REPLACE" # To be replaced (API keys to be found in google docs)
        }
        
        # Send the API request
        response = requests.get(base_url, params=params)
        data = response.json()

        # Variables for counting headlines and storing unique ones
        count = 0
        headline = ""
        
        # Process the response data
        for i in range(1, 50):
             # Check if the stock is present in the title and it is not a duplicate headline
            if stock in data['feed'][i-1:i][0]['title'] and data['feed'][i-1:i][0]['title'] not in headline and data['feed'][i-1:i][0]['title'] not in list_of_headlines:
                # Append the headline and its link to the result string
                headline = headline + data['feed'][i-1:i][0]['title'] + "- Link: " + data['feed'][i-1:i][0]['url'] + 2*"\n"
                count = count + 1
            # Break the loop if three headlines have been collected
            if count == 3:
                break

        # Check for errors in the response
        if "Error Message" in data:
            print(f"Error retrieving data for stock: {stock}")
            continue
        
        # Add the stock and its headlines to the result string
        if headline not in list_of_headlines:
            list_of_headlines = list_of_headlines + stock + ":" + "\n" + headline + 3*"\n"
    
    # Return the final list of headlines
    return list_of_headlines

# GUI Functions 

In [27]:
# Creating a function for portfolio optimization based on stock input
def submit():
    global canvas
    output_text.delete("1.0", tk.END)  # Clears the output text

    if canvas:
        # Clears the old graph from the canvas in the interface
        canvas.get_tk_widget().pack_forget()
        canvas.get_tk_widget().destroy()
        
    stocks_value = input_stocks_entry.get()
    global stocks, pf_data, mean_returns, cov_matrix, risk_free_rate, num_portfolios, money_to_invest, scenario 
    
    # Preventing comma issue at the beginning or end of stock input
    stocks_with_commas = stocks_value.upper().replace(" ", "")
    stocks = [stock.strip() for stock in stocks_with_commas.split(",") if stock.strip()]

        
    
    #Creating dataFrame with relevant stocks/bonds
    pf_data = pd.DataFrame()

    # Pulling closing price of stocks  
    for stock in stocks:
        stock_data = yf.download(stock, start='2008-01-01')
        if stock_data.empty:
            output_text.insert(tk.END, f"No data found for stock, check the ticker: {stock}\n")
            return
        pf_data[stock] = stock_data['Adj Close']
    
    for stock in stocks:
        pf_data[stock] = ata = yf.download(stock,'2008-1-1')['Adj Close']
        
    # Stock typing might be issue, no comma on end
    # Base Parameters
    returns = pf_data.pct_change()
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    
    # Get the current interest rate
    risk_free_rate = get_current_interest_rate()
    if risk_free_rate == None:
        output_text.insert(tk.END, f"Current Federal Funds rate could not be retrieved\n")
        return
    num_portfolios = 100000
    
    # Setting the amount to invest
    try:
        money_to_invest = float(moneyInv_entry.get())
    except (ValueError):
        output_text.insert(tk.END, f"Please input valid amount to invest\n")
        return
    
    # Assigning numbers for each optimization strategy
    exclusive_checkbox1_value = 1 if exclusive_checkbox1_var.get() == 1 else 0
    exclusive_checkbox2_value = 2 if exclusive_checkbox2_var.get() == 1 else 0
    scenario = exclusive_checkbox1_value + exclusive_checkbox2_value
    
    # Creating conditional statements to return error messages for wrong input
    if exclusive_checkbox1_var.get() + exclusive_checkbox2_var.get() != 1:
        output_text.insert(tk.END, f"Please only select exactly one checkbox from each subsection\n")
        return
    else:
        output_generated = display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate, scenario, money_to_invest)
        output_text.insert(tk.END, output_generated[0])
        canvas = FigureCanvasTkAgg(output_generated[1], master=root)
        canvas.get_tk_widget().configure(width=600, height=600)
        canvas.draw()
        canvas.get_tk_widget().pack()
        global flip
        flip = True # determine whether first function was executed already
        
# Defining function that uses the get_stocj_headlines() function 
def news():
    output_text.delete("1.0", tk.END)
    stocks_value = input_stocks_entry.get()
    
    # Preventing comma issue at the beginning or end of stock input
    stocks_with_commas = stocks_value.upper().replace(" ", "")
    global stocks
    stocks = [stock.strip() for stock in stocks_with_commas.split(",") if stock.strip()]
    news_text = get_stock_headlines(stocks)
    output_text.insert(tk.END, news_text)
    
# Defining function that calculates Value at Risk    
def get_var():
    output_text.delete("1.0", tk.END)
    #Retrieving values from input fields
    try:
        conf = float(inputInterval_entry.get())
        conf_int = 1 - conf
    except (ValueError):
        output_text.insert(tk.END, f"Please input a valid confidence interval\n")
        return
    try:
        days = float(inputDays_entry.get())
    except (ValueError):
        output_text.insert(tk.END, f"Please input a valid number of days\n")
        return
    
   
    # Creating conditional statements to return error messages for wrong input
    if conf_int <= 0 or conf_int >= 0.5:
        output_text.insert(tk.END, f"Please select a valid confidence interval between 0 and 1\n")
        return
    elif flip == True:
        if scenario == 1:
            daily_sdp = sdp / (252 ** 2)
            total_sdp = daily_sdp * (days ** 2) * money_to_invest
            cutoff = max(norm.ppf(conf_int, money_to_invest, total_sdp), 0)
            valueatrisk = "The total value at Risk is: " + str(money_to_invest - cutoff) + "$"
            output_text.insert(tk.END, valueatrisk)
        elif scenario == 2:   
            daily_sdp_min = sdp_min / (252 ** 2)
            total_sdp_min = daily_sdp_min * (days ** 2) * money_to_invest
            cutoff = max(norm.ppf(conf_int, money_to_invest, total_sdp_min), 0)
            valueatrisk = "The total value at Risk is: " + str(money_to_invest - cutoff) + "$"
            output_text.insert(tk.END, valueatrisk)

# GUI Interface

In [30]:
canvas = None
# Creating the GUI
root = tk.Tk()
root.geometry("600x1200")
root.title("Stock Portfolio Manager")

# Setting headline
exclusive_label = tk.Label(root, text="Best Stock allocation", font=('Arial 20'), anchor="w")
exclusive_label.pack(anchor="w")

# Creating stock ticker input
input_stocks_label = tk.Label(root, text="Stock Tickers (Separated by Commas)", font=('Arial 14'), anchor="w")
input_stocks_label.pack(anchor="w")
input_stocks_entry = tk.Entry(root)
input_stocks_entry.pack(anchor="w")

# Creating money to invest input
moneyInv_label = tk.Label(root, text="Total Money to Invest (in $):", font=('Arial 14'), anchor="w")
moneyInv_label.pack(anchor="w")
moneyInv_entry = tk.Entry(root)
moneyInv_entry.pack(anchor="w")

exclusive_label = tk.Label(root, text="Optimization Strategy:", font=('Arial 14'), anchor="w")
exclusive_label.pack(anchor="w")

#Creating buttons for the different optimization stategies
exclusive_checkbox1_var = tk.IntVar()
exclusive_checkbox1 = tk.Checkbutton(root, text="Optimal Risk and Return Combination", variable=exclusive_checkbox1_var, font=('Arial 14'), anchor="w")
exclusive_checkbox1.pack(anchor="w")

exclusive_checkbox2_var = tk.IntVar()
exclusive_checkbox2 = tk.Checkbutton(root, text="Lowest Risk", variable=exclusive_checkbox2_var, font=('Arial 14'), anchor="w")
exclusive_checkbox2.pack(anchor="w")

#Creating submit button
submitPortfolio_button = tk.Button(root, text="Submit", command=submit, font=('Arial 14'))
submitPortfolio_button.pack(anchor="w")

# Setting headline
exclusive_label = tk.Label(root, text="Value at Risk:", font=('Arial 20'), anchor="w")
exclusive_label.pack(anchor="w")

# Creating confodence interval input
inputInterval_label = tk.Label(root, text="Confidence Interval (between 0.5 and 1):", font=('Arial 14'), anchor="w")
inputInterval_label.pack(anchor="w")
inputInterval_entry = tk.Entry(root)
inputInterval_entry.pack(anchor="w")

# Creating days interval input
inputDays_label = tk.Label(root, text="Period (in Days):", font=('Arial 14'), anchor="w")
inputDays_label.pack(anchor="w")
inputDays_entry = tk.Entry(root)
inputDays_entry.pack(anchor="w")
 # Creating submit button
submitVaR_button = tk.Button(root, text="Get VaR", command=get_var, font=('Arial 14'))
submitVaR_button.pack(anchor="w")

frame1 = tk.Frame(root)
frame1.pack()

# Setting headline
exclusive_label = tk.Label(root, text="Latest News:", font=('Arial 20'), anchor="w")
exclusive_label.pack(anchor="w")

# Creating submit button
submitNews_button = tk.Button(root, text="Get Latest Relevant News", command=news, font=('Arial 14'))
submitNews_button.pack(anchor="w")

# creating output window
output_text = tk.Text(root, height=8, width=200)
output_text.pack(anchor="w")

root.mainloop()